In [ ]:
#check BIOGEME version
import biogeme.version as ver
print("Biogeme version:", ver.get_version())

#Import necessary libraries
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable

In [ ]:
# Load the database
df = pd.read_csv("apollo_modeChoiceData.csv", sep=',') # Check the path to your CSV file

#create backup of the original dataframe
df_backup = df.copy()
df

df['income'] = df['income'] / 1000  # Scale income to thousands

In [ ]:
# Check missing values
print(df.isnull().sum())

In [ ]:
# Drop columns with missing values
## df = df.drop(columns=["SP_task"])
# Or, set the missing values to -1
df = df.fillna(-1)
print(df.isnull().sum())
# Additional preprocessing the data
# Create a row index
df["RowID"] = df.index + 1
# Define the BIOGEME-specific database
database = db.Database("ModeChoice", df)

In [ ]:
"""
Use the loop below to get all of columns names.
for col in df.columns:
    print(f'{col} = Variable("{col}")')

1. The code will generate the Variable for each column.    
2. Copy the output and paste it below to define the variables.
"""
for col in df.columns:
    print(f'{col} = Variable("{col}")')

In [ ]:
ID = Variable("ID")
RP = Variable("RP")
SP = Variable("SP")
RP_journey = Variable("RP_journey")
SP_task = Variable("SP_task")
av_car = Variable("av_car")
av_bus = Variable("av_bus")
av_air = Variable("av_air")
av_rail = Variable("av_rail")
time_car = Variable("time_car")
cost_car = Variable("cost_car")
time_bus = Variable("time_bus")
cost_bus = Variable("cost_bus")
access_bus = Variable("access_bus")
time_air = Variable("time_air")
cost_air = Variable("cost_air")
access_air = Variable("access_air")
service_air = Variable("service_air")
time_rail = Variable("time_rail")
cost_rail = Variable("cost_rail")
access_rail = Variable("access_rail")
service_rail = Variable("service_rail")
female = Variable("female")
business = Variable("business")
income = Variable("income")
choice = Variable("choice")
RowID = Variable("RowID")

In [ ]:
# Define beta values to be estimated
## Beta('name', initial value, lower bound, upper bound, reference)
## Reference is used to set the reference alternative in a logit model (1 = yes, 0 = no)
asc_car = Beta('asc_car', 0, None, None, 1) #Reference alternatives
asc_bus = Beta('asc_bus', 0, None, None, 0)
asc_air = Beta('asc_air', 0, None, None, 0)
asc_rail = Beta('asc_rail', 0, None, None, 0) 

b_cost_car = Beta('b_cost_car', 0, None, None, 0)
b_cost_bus = Beta('b_cost_bus', 0, None, None, 0)
b_cost_air = Beta('b_cost_air', 0, None, None, 0)
b_cost_rail = Beta('b_cost_rail', 0, None, None, 0)

b_time = Beta('b_time', 0, None, None, 0)
b_income = Beta('b_income', 0, None, None, 0)
b_inc_cost = Beta('b_inc_cost', 0, None, None, 0)

# Utility functions
V_car = (asc_car + 
         b_cost_car * cost_car + 
         b_time * time_car + 
         b_income * income + 
         b_inc_cost * (cost_car/income)
        )

V_bus = (asc_bus +
            b_cost_bus * cost_bus + 
            b_time * time_bus + 
            b_income * income + 
            b_inc_cost * (cost_bus/income)    
        )

V_air = (asc_air +
          b_cost_air * cost_air + 
          b_time * time_air + 
          b_income * income + 
          b_inc_cost * (cost_air/income)    
        )

V_rail = (asc_rail +
          b_cost_rail * cost_rail + 
          b_time * time_rail + 
          b_income * income + 
          b_inc_cost * (cost_rail/income)    
        )


# Dictionary defining alternatives mapping
V = {1: V_car, 2: V_bus, 3: V_air, 4: V_rail}

# Dictionary defining availability mapping
av = {1: av_car, 2: av_bus,3: av_air, 4: av_rail}

# Define the choice model
# loglogit(Alternatives mapping, Availability mapping, Choice variable)
logprob= models.loglogit(V, av, choice)

# Estimate Model
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.model_name = 'MNL_model' # Set the model name

#Calculate null Loglikelihood
the_biogeme.calculate_null_loglikelihood(av)

# Save the estimation results
MNL_model = the_biogeme.estimate()

#Print the results
print(MNL_model.short_summary())
MNL_model.get_estimated_parameters()

In [ ]:
#computing the choice probabilities for each row in the database
simulate = {
    'prob_car': models.logit(V, av, 1),  # P(choice=1)
    'prob_bus': models.logit(V, av, 2),  # P(choice=2)
    'prob_air': models.logit(V, av, 3),  # P(choice=3)
    'prob_rail': models.logit(V, av, 4)  # P(choice=4)
}

biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(MNL_model.get_beta_values())
simulated_values

## Derivatives and Elasticities

*DERIVATIVES*

The change in $P_{ni}$ as attribute $X_{ni}$ changes by 1.
$$\frac{\partial P_{ni}}{\partial X_{ni}} = \frac{\partial V_{ni}}{\partial X_{ni}} P_{ni}\left(1-P_{ni}\right) = \beta_i^{X}P_{ni}\left(1-P_{ni}\right)$$

The change in $P_{ni}$ as attribute $X_{nj}$ changes by 1.
$$\frac{\partial P_{ni}}{\partial X_{nj}} = - \frac{\partial V_{nj}}{\partial X_{nj}} P_{ni}P_{nj} = - \beta_j^{X}P_{ni}P_{nj} $$

In [9]:
MNL_model.get_beta_values()

{'b_cost_car': -0.01630193904590192,
 'b_time': -0.010029922450244031,
 'b_income': -5.111718938564405e-12,
 'b_inc_cost': -1.1432664762153948,
 'asc_bus': -2.025906578250442,
 'b_cost_bus': -0.019986562506533807,
 'asc_air': -0.9356203966270109,
 'b_cost_air': -0.017735969042145482,
 'asc_rail': 0.04664074822529534,
 'b_cost_rail': -0.02903245194274424}

In [ ]:
beta_time = MNL_model.get_beta_values()["b_time"]
#use 100th prediction
prob_car_100 = float(simulated_values.iloc[99].prob_car)
prob_bus_100 = float(simulated_values.iloc[99].prob_bus)
prob_air_100 = float(simulated_values.iloc[99].prob_air)
prob_rail_100 = float(simulated_values.iloc[99].prob_rail)
print("Probabilities for the 100th observation:")
print(f'P(car) = {prob_car_100:4f}')
print(f'P(bus) = {prob_bus_100:4f}')
print(f'P(air) = {prob_air_100:4f}')
print(f'P(rail) = {prob_rail_100:4f}')

In [ ]:
#Change of P(air) as air travel time increases by 1
derivatives_air = MNL_model.get_beta_values()["b_time"] * prob_air_100 * (1 - prob_air_100)
#Change of P(car) as air travel time increases by 1
derivatives_car = MNL_model.get_beta_values()["b_time"] * prob_car_100 * prob_air_100
#Change of P(bus) as air travel time increases by 1
derivatives_bus = MNL_model.get_beta_values()["b_time"] * prob_bus_100 * prob_air_100
#Change of P(rail) as air travel time increases by 1
derivatives_rail = MNL_model.get_beta_values()["b_time"] * prob_rail_100 * prob_air_100

print(f'Derivative of P(air) w.r.t time_air: {derivatives_air}')
print(f'Derivative of P(car) w.r.t time_air: {derivatives_car}')
print(f'Derivative of P(bus) w.r.t time_air: {derivatives_bus}')
print(f'Derivative of P(rail) w.r.t time_air: {derivatives_rail}')

**Elasticities**

Direct Elasticities:

Proportional change of $P_{ni}$ due to change in $X_{ni}$:
$$\mathcal{E}_{ni}^{X_{ni}} = \frac{\frac{P_{ni}^{v2}-P_{ni}^{v1}}{P_{ni}^{v1}}}{\frac{X_{ni}^{v2}-X_{ni}^{v1}}{X_{ni}^{v1}}} =\frac{\partial P_{ni}}{\partial X_{ni}} \frac{ X_{ni}}{ P_{ni} } = \beta_i^{X}X_{ni}\left(1-P_{ni}\right)$$

Cross Elasticities:

Proportional change of $P_{ni}$ due to change in $X_{nj}$:
$$\mathcal{E}_{ni}^{X_{nj}} = - \beta_j^{X}X_{nj}P_{nj} $$

In [ ]:
beta_cost_car = MNL_model.get_beta_values()["b_cost_car"]

#Direct elasticity w.r.t cost_car
elasticity_car = beta_cost_car * cost_car_100 * (1 - prob_car_100)

#Cross elasticity w.r.t cost_car
elasticity_bus = - beta_cost_car * cost_car_100 * prob_bus_100
elasticity_air = - beta_cost_car * cost_car_100 * prob_air_100
elasticity_rail = - beta_cost_car * cost_car_100 * prob_rail_100

print(f'Elasticity of P(car) w.r.t cost_car: {elasticity_car}')
print(f'Elasticity of P(bus) w.r.t cost_car: {elasticity_bus}')
print(f'Elasticity of P(air) w.r.t cost_car: {elasticity_air}')
print(f'Elasticity of P(rail) w.r.t cost_car: {elasticity_rail}')